In [1]:
import numpy as np
from astropy.table import Table

We use the constants for different IA models as inputs

In [2]:
A_IA = 1
b_TA = 0
C_2= 0
##I think I'm forgetting something here

The intensity of the NLA, delta-NLA and TT models can be chosen by tweaking the numbers above, this can probably be changed for some calculation involving weights given to each model but that's up for discussion, for now, we setup for the observed epsilons with and without noise, using the provided parameters

In [3]:
for i in np.arange(5)+1:
    catalog = Table.read('/global/homes/j/jharno/IA-infusion/SkySim5000/GalCat/StageIV_nz/GalCat_tomo{}_All_0.06GpAM_RA_Dec_g1_g2_w_den_Sij.dat'.format(i),
                     format='ascii')  # descriptions=['ra_deg', 'dec_deg', 'shear1', 'shear2', 'weight', 'z_true', 'delta', 's11', 's22', 's12'])
    epsilons = Table.read('/global/cscratch1/sd/dlan/SkySim5000_IA_infusion/epsilon_{}.dat'.format(i), format='ascii')  #descriptions=['e1','e2','e1_noise','e2_noise']
    
    ra=np.asarray(catalog['col1'])
    dec=np.asarray(catalog['col2'])
    weight=np.asarray(catalog['col5'])
    z=np.asarray(catalog['col6'])
    gamma1 = np.asarray(catalog['col3'])
    gamma2 = np.asarray(catalog['col4'])
    e1_NLA = np.asarray(epsilons['e1_NLA'])
    e2_NLA = np.asarray(epsilons['e2_NLA'])
    e1_TT = np.asarray(epsilons['e1_TT'])
    e2_TT = np.asarray(epsilons['e2_TT'])
    e1_noise = np.asarray(epsilons['e1_noise'])
    e2_noise = np.asarray(epsilons['e2_noise'])
    delta = np.asarray(catalog['col7'])
    epsilon_NLA = np.empty(e1_NLA.shape, dtype = complex)
    epsilon_NLA.real = A_IA * e1_NLA  
    epsilon_NLA.imag = A_IA * e2_NLA
    epsilon_TT = np.empty(e1_TT.shape, dtype = complex)
    epsilon_TT.real = C_2 * e1_TT  
    epsilon_TT.imag = C_2 * e2_TT
    epsilon_random = np.empty(e1_TT.shape, dtype = complex)
    epsilon_random.real = e1_noise
    epsilon_random.imag = e2_noise
    g = np.empty(gamma1.shape, dtype=complex)
    g.real = gamma1
    g.imag = gamma2
    #And finally, we calculate the observed epsilons using the summation as in equation 11 of the paper
    epsilon_IA_TATT = epsilon_NLA * (1 + b_TA * delta) + epsilon_TT
    epsilon_int = (epsilon_IA_TATT + epsilon_random)/(1 + np.conjugate(epsilon_IA_TATT) * epsilon_random)
    epsilon_obs = (epsilon_int + g)/(1 + np.conjugate(epsilon_int) * g)
    epsilon_obs_no_noise = (epsilon_IA_TATT + g)/(1 + epsilon_IA_TATT * np.conjugate(g))
   
    fin_epsilon = Table([ra,dec,z,weight,delta,epsilon_obs.real,epsilon_obs.imag,epsilon_obs_no_noise.real,epsilon_obs_no_noise.imag,epsilon_IA_TATT.real,epsilon_IA_TATT.imag], names = ('ra','dec','z','weight','delta','e_obs_1','e_obs_2', 'e_obs_no_noise_1','e_obs_no_noise_2','e_IA_TATT_1', 'e_IA_TATT_2' ))
    fin_epsilon.write('/global/cscratch1/sd/dlan/SkySim5000_IA_infusion/fin_epsilon_IA_{}.dat'.format(i), format='ascii', overwrite=True)